In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.layers import LSTM, BatchNormalization,Input
from tensorflow.keras import Model
from sklearn.preprocessing import MinMaxScaler
import datetime

2021-08-09 04:27:11.765632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-09 04:27:11.765692: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# 원본데이터
train = pd.read_csv('./flask/app/data/train_data.csv')

In [3]:
train.columns = ['시간', 'id', 'sub_id', '발전량', '발전용량', '위도', '경도', '기온', '강수량', '풍속', '풍향', '습도', '전운량']

In [4]:
# plant 19 20 21 발전량을 1000으로 나누기(단위 변환)

train_true = train.loc[(train['id']!='plant19') & (train['id']!='plant20') & (train['id']!='plant21')]
train['id'].unique()

array(['plant1', 'plant2', 'plant3', 'plant4', 'plant5', 'plant6',
       'plant7', 'plant8', 'plant9', 'plant10', 'plant11', 'plant12',
       'plant13', 'plant14', 'plant15', 'plant16', 'plant17', 'plant18',
       'plant19', 'plant20', 'plant21', 'plant22', 'plant23', 'plant24',
       'plant25', 'plant26', 'plant27', 'plant30', 'plant32'],
      dtype=object)

In [5]:
# 19, 20, 21 발전소의 경우 발전량 측정 단위가 다르기 때문에 1000으로 나눠주기
train_false = train.loc[(train['id']=='plant19') | (train['id']=='plant20') | (train['id']=='plant21')]
train_false['발전량'] = train_false['발전량'] / 1000

<ipython-input-5-94d7b4f3cb8d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_false['발전량'] = train_false['발전량'] / 1000


In [6]:
train_false['id'].unique()

array(['plant19', 'plant20', 'plant21'], dtype=object)

In [7]:
# 데이터 concat(단위 잘못된 plant 수정 후 concat)
data = pd.concat([train_true, train_false]).reset_index(drop=True)
data

,시간,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량
0,2018-01-01 00:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.4,0.0,12.5,340.0,56.0,NaN
1,2018-01-01 01:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.7,0.0,10.9,340.0,60.0,NaN
2,2018-01-01 02:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,9.8,340.0,56.0,NaN
3,2018-01-01 03:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,10.8,360.0,59.0,NaN
4,2018-01-01 04:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
626947,2020-06-30 19:00:00,plant21,1,48.0,13295.0,34.73819,126.41546,19.9,0.0,5.9,290.0,87.0,2.0
626948,2020-06-30 20:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.9,0.3,5.6,320.0,95.0,10.0
626949,2020-06-30 21:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.6,0.6,7.1,340.0,97.0,10.0
626950,2020-06-30 22:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.3,0.1,6.7,340.0,97.0,10.0


In [8]:
# 년, 월, 일, 시간 추출
data = data.copy()
data['시간'] = pd.to_datetime(data['시간'])
data = data.rename(columns={'시간':'Date'})

data['년도'] = data['Date'].dt.year
data['월'] = data['Date'].dt.month
data['일'] = data['Date'].dt.day
data['시간'] = data['Date'].dt.hour

data

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
0,2018-01-01 00:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.4,0.0,12.5,340.0,56.0,NaN,2018,1,1,0
1,2018-01-01 01:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.7,0.0,10.9,340.0,60.0,NaN,2018,1,1,1
2,2018-01-01 02:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,9.8,340.0,56.0,NaN,2018,1,1,2
3,2018-01-01 03:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,10.8,360.0,59.0,NaN,2018,1,1,3
4,2018-01-01 04:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN,2018,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626947,2020-06-30 19:00:00,plant21,1,48.0,13295.0,34.73819,126.41546,19.9,0.0,5.9,290.0,87.0,2.0,2020,6,30,19
626948,2020-06-30 20:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.9,0.3,5.6,320.0,95.0,10.0,2020,6,30,20
626949,2020-06-30 21:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.6,0.6,7.1,340.0,97.0,10.0,2020,6,30,21
626950,2020-06-30 22:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.3,0.1,6.7,340.0,97.0,10.0,2020,6,30,22


In [9]:
# 중복되는 행 제거
data = data.drop_duplicates(['id','sub_id','Date'], keep='first').reset_index(drop=True)
data

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
0,2018-01-01 00:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.4,0.0,12.5,340.0,56.0,NaN,2018,1,1,0
1,2018-01-01 01:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.7,0.0,10.9,340.0,60.0,NaN,2018,1,1,1
2,2018-01-01 02:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,9.8,340.0,56.0,NaN,2018,1,1,2
3,2018-01-01 03:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.5,0.0,10.8,360.0,59.0,NaN,2018,1,1,3
4,2018-01-01 04:00:00,plant1,1,0.0,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN,2018,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626899,2020-06-30 19:00:00,plant21,1,48.0,13295.0,34.73819,126.41546,19.9,0.0,5.9,290.0,87.0,2.0,2020,6,30,19
626900,2020-06-30 20:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.9,0.3,5.6,320.0,95.0,10.0,2020,6,30,20
626901,2020-06-30 21:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.6,0.6,7.1,340.0,97.0,10.0,2020,6,30,21
626902,2020-06-30 22:00:00,plant21,1,0.0,13295.0,34.73819,126.41546,18.3,0.1,6.7,340.0,97.0,10.0,2020,6,30,22


In [10]:
# 발전량이 없는 시간대 제외
hour_limit = data[(data['시간'] > 3) & (data['시간'] < 20 )].reset_index(drop=True)
hour_limit

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
0,2018-01-01 04:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN,2018,1,1,4
1,2018-01-01 05:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.4,0.0,8.9,360.0,64.0,NaN,2018,1,1,5
2,2018-01-01 06:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,8.7,360.0,58.0,NaN,2018,1,1,6
3,2018-01-01 07:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,7.0,360.0,61.0,NaN,2018,1,1,7
4,2018-01-01 08:00:00,plant1,1,15.476,196.0,33.23831,126.34047,5.8,0.0,8.1,360.0,59.0,NaN,2018,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417931,2020-06-30 15:00:00,plant21,1,3043.200,13295.0,34.73819,126.41546,21.9,0.0,6.6,320.0,78.0,8.0,2020,6,30,15
417932,2020-06-30 16:00:00,plant21,1,3734.400,13295.0,34.73819,126.41546,21.5,0.0,6.9,290.0,80.0,8.0,2020,6,30,16
417933,2020-06-30 17:00:00,plant21,1,1958.400,13295.0,34.73819,126.41546,21.0,0.0,6.4,320.0,81.0,5.0,2020,6,30,17
417934,2020-06-30 18:00:00,plant21,1,662.400,13295.0,34.73819,126.41546,20.4,0.0,5.7,320.0,86.0,7.0,2020,6,30,18


In [11]:
# 발전량이 결측치인 부분 삭제하기
nan_target = hour_limit[hour_limit['발전량'].isnull()]
nan_target_index = nan_target.index
sample = hour_limit.drop(nan_target_index).reset_index(drop=True)
sample

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
0,2018-01-01 04:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN,2018,1,1,4
1,2018-01-01 05:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.4,0.0,8.9,360.0,64.0,NaN,2018,1,1,5
2,2018-01-01 06:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,8.7,360.0,58.0,NaN,2018,1,1,6
3,2018-01-01 07:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,7.0,360.0,61.0,NaN,2018,1,1,7
4,2018-01-01 08:00:00,plant1,1,15.476,196.0,33.23831,126.34047,5.8,0.0,8.1,360.0,59.0,NaN,2018,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415914,2020-06-30 15:00:00,plant21,1,3043.200,13295.0,34.73819,126.41546,21.9,0.0,6.6,320.0,78.0,8.0,2020,6,30,15
415915,2020-06-30 16:00:00,plant21,1,3734.400,13295.0,34.73819,126.41546,21.5,0.0,6.9,290.0,80.0,8.0,2020,6,30,16
415916,2020-06-30 17:00:00,plant21,1,1958.400,13295.0,34.73819,126.41546,21.0,0.0,6.4,320.0,81.0,5.0,2020,6,30,17
415917,2020-06-30 18:00:00,plant21,1,662.400,13295.0,34.73819,126.41546,20.4,0.0,5.7,320.0,86.0,7.0,2020,6,30,18


In [12]:
# 발전량이 없는 시간대 제외하면 하루에 16행이 나와야 정상
check = pd.DataFrame(sample.groupby(['id', 'sub_id','년도', '월', '일'])['id'].count())
check.columns=['count']
check = check.reset_index()
check = check.loc[check['count']!=16]

In [13]:
# 해당 plant20의 2020년도 1월 22일은 1행이 결측치
check

,id,sub_id,년도,월,일,count
15877,plant20,1,2020,1,22,15


In [14]:
df = pd.DataFrame()
for i in range(len(check)):
    ID = sample[sample['id']==check['id'].iloc[i]]
    sub_id = ID[ID['sub_id']==check['sub_id'].iloc[i]]
    year = sub_id[sub_id['년도']==check['년도'].iloc[i]]  
    month = year[year['월']==check['월'].iloc[i]]
    day = month[month['일']==check['일'].iloc[i]]              
    df = pd.concat([df, day], axis=0)
    
df

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
398752,2020-01-22 04:00:00,plant20,1,0.0,1490.0,37.38136,126.81703,-0.5,0.0,0.6,290.0,79.0,10.0,2020,1,22,4
398753,2020-01-22 05:00:00,plant20,1,0.0,1490.0,37.38136,126.81703,-0.5,0.0,0.0,0.0,80.0,10.0,2020,1,22,5
398754,2020-01-22 06:00:00,plant20,1,0.0,1490.0,37.38136,126.81703,0.2,0.0,0.1,0.0,75.0,10.0,2020,1,22,6
398755,2020-01-22 07:00:00,plant20,1,0.0,1490.0,37.38136,126.81703,0.5,0.0,0.9,20.0,69.0,10.0,2020,1,22,7
398756,2020-01-22 08:00:00,plant20,1,7.2,1490.0,37.38136,126.81703,0.9,0.0,0.5,110.0,60.0,10.0,2020,1,22,8
398757,2020-01-22 09:00:00,plant20,1,61.2,1490.0,37.38136,126.81703,2.1,0.0,1.3,90.0,51.0,10.0,2020,1,22,9
398758,2020-01-22 11:00:00,plant20,1,172.8,1490.0,37.38136,126.81703,3.5,0.0,0.6,90.0,45.0,10.0,2020,1,22,11
398759,2020-01-22 12:00:00,plant20,1,310.8,1490.0,37.38136,126.81703,3.8,0.0,1.9,160.0,59.0,10.0,2020,1,22,12
398760,2020-01-22 13:00:00,plant20,1,327.6,1490.0,37.38136,126.81703,3.2,0.0,3.1,140.0,78.0,10.0,2020,1,22,13
398761,2020-01-22 14:00:00,plant20,1,531.6,1490.0,37.38136,126.81703,3.3,0.0,2.3,140.0,85.0,10.0,2020,1,22,14


In [15]:
# 해당 행 원데이터에서 삭제

df_index = df.index
sample = sample.drop(df_index).reset_index(drop=True)
    
sample

,Date,id,sub_id,발전량,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,전운량,년도,월,일,시간
0,2018-01-01 04:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,NaN,2018,1,1,4
1,2018-01-01 05:00:00,plant1,1,0.000,196.0,33.23831,126.34047,6.4,0.0,8.9,360.0,64.0,NaN,2018,1,1,5
2,2018-01-01 06:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,8.7,360.0,58.0,NaN,2018,1,1,6
3,2018-01-01 07:00:00,plant1,1,0.000,196.0,33.23831,126.34047,5.9,0.0,7.0,360.0,61.0,NaN,2018,1,1,7
4,2018-01-01 08:00:00,plant1,1,15.476,196.0,33.23831,126.34047,5.8,0.0,8.1,360.0,59.0,NaN,2018,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415899,2020-06-30 15:00:00,plant21,1,3043.200,13295.0,34.73819,126.41546,21.9,0.0,6.6,320.0,78.0,8.0,2020,6,30,15
415900,2020-06-30 16:00:00,plant21,1,3734.400,13295.0,34.73819,126.41546,21.5,0.0,6.9,290.0,80.0,8.0,2020,6,30,16
415901,2020-06-30 17:00:00,plant21,1,1958.400,13295.0,34.73819,126.41546,21.0,0.0,6.4,320.0,81.0,5.0,2020,6,30,17
415902,2020-06-30 18:00:00,plant21,1,662.400,13295.0,34.73819,126.41546,20.4,0.0,5.7,320.0,86.0,7.0,2020,6,30,18


In [18]:
# 기온 결측치 년도, 해당월 중앙값으로 결측치 처리
for i in sample['id'].unique(): 
    for j in sample[sample['id'] == i]['sub_id'].unique():
        null = sample[(sample['id'] == i) & (sample['sub_id']==j)]
        tem_null = null.loc[null['기온'].isnull()]
        
        for k in range(len(tem_null)):
            year = tem_null['년도'].iloc[k]
            month = tem_null['월'].iloc[k]
            hour = tem_null['시간'].iloc[k]
            
            month_data = null.loc[(null['년도'] == year) & (null['월'] == month) & (null['시간'] == hour)]
            temp_median = month_data['기온'].median()
            sample.loc[(sample['id'] == i) & (sample['sub_id'] == j) &(sample['년도'] == year) & (sample['월'] == month) & 
                       (sample['시간'] == hour), '기온'] = temp_median

In [19]:
# 풍속 결측치 년도, 해당월 중앙값으로 결측치 처리
for i in sample['id'].unique(): 
    for j in sample[sample['id'] == i]['sub_id'].unique():
        null = sample[(sample['id'] == i) & (sample['sub_id']==j)]
        tem_null = null.loc[null['풍속'].isnull()]
        
        for k in range(len(tem_null)):
            year = tem_null['년도'].iloc[k]
            month = tem_null['월'].iloc[k]
            hour = tem_null['시간'].iloc[k]
            
            month_data = null.loc[(null['년도'] == year) & (null['월'] == month) & (null['시간'] == hour)]
            temp_median = month_data['풍속'].median()
            sample.loc[(sample['id'] == i) & (sample['sub_id'] == j) &(sample['년도'] == year) & (sample['월'] == month) & 
                       (sample['시간'] == hour), '풍속'] = temp_median

In [20]:
# 풍향 결측치 년도, 해당월 중앙값으로 결측치 처리
for i in sample['id'].unique(): 
    for j in sample[sample['id'] == i]['sub_id'].unique():
        null = sample[(sample['id'] == i) & (sample['sub_id']==j)]
        tem_null = null.loc[null['풍향'].isnull()]
        
        for k in range(len(tem_null)):
            year = tem_null['년도'].iloc[k]
            month = tem_null['월'].iloc[k]
            hour = tem_null['시간'].iloc[k]
            
            month_data = null.loc[(null['년도'] == year) & (null['월'] == month) & (null['시간'] == hour)]
            temp_median = month_data['풍향'].median()
            sample.loc[(sample['id'] == i) & (sample['sub_id'] == j) &(sample['년도'] == year) & (sample['월'] == month) & 
                       (sample['시간'] == hour), '풍향'] = temp_median

In [21]:
# 습도 결측치 년도, 해당월 중앙값으로 결측치 처리
for i in sample['id'].unique(): 
    for j in sample[sample['id'] == i]['sub_id'].unique():
        null = sample[(sample['id'] == i) & (sample['sub_id']==j)]
        tem_null = null.loc[null['습도'].isnull()]
        
        for k in range(len(tem_null)):
            year = tem_null['년도'].iloc[k]
            month = tem_null['월'].iloc[k]
            hour = tem_null['시간'].iloc[k]
            
            month_data = null.loc[(null['년도'] == year) & (null['월'] == month) & (null['시간'] == hour)]
            temp_median = month_data['습도'].median()
            sample.loc[(sample['id'] == i) & (sample['sub_id'] == j) &(sample['년도'] == year) & (sample['월'] == month) & 
                       (sample['시간'] == hour), '습도'] = temp_median

In [22]:
# 기상 데이터 결측치가 존재
sample.isnull().sum()

Date          0
id            0
sub_id        0
발전량           0
발전용량          0
위도            0
경도            0
기온         7977
강수량           0
풍속         7977
풍향         7977
습도         7977
전운량       57349
년도            0
월             0
일             0
시간            0
dtype: int64

In [23]:
# 해당 plant19의 경우 기상 데이터 결측치가 존재하여 삭제
sample = sample.loc[sample['id'] != 'plant19'].reset_index(drop=True)
sample.isnull().sum()

Date          0
id            0
sub_id        0
발전량           0
발전용량          0
위도            0
경도            0
기온            0
강수량           0
풍속            0
풍향            0
습도            0
전운량       44634
년도            0
월             0
일             0
시간            0
dtype: int64

In [25]:
sample = sample[['id', 'sub_id',  '발전용량', '위도', '경도', '기온', '강수량', '풍속', '풍향',
       '습도', '년도', '월', '일', '시간', '발전량' ]]

In [26]:
# 하루 발전량 평균이 0인 행 삭제
check_zero = pd.DataFrame(sample.groupby(['id', 'sub_id','년도', '월', '일'])['발전량'].mean()).reset_index()
check_zero = check_zero[check_zero['발전량']==0]
check_zero

,id,sub_id,년도,월,일,발전량
234,plant1,1,2018,8,23,0.0
404,plant1,1,2019,2,9,0.0
405,plant1,1,2019,2,10,0.0
406,plant1,1,2019,2,11,0.0
407,plant1,1,2019,2,12,0.0
...,...,...,...,...,...,...
24128,plant8,1,2020,7,21,0.0
24129,plant8,1,2020,7,22,0.0
24130,plant8,1,2020,7,23,0.0
24525,plant9,1,2019,1,22,0.0


In [27]:
# 중복 체크
dfr = pd.DataFrame()
for i in range(len(check_zero)):
    ID = sample[sample['id']==check_zero['id'].iloc[i]]
    sub_id = ID[ID['sub_id']==check_zero['sub_id'].iloc[i]]
    year = sub_id[sub_id['년도']==check_zero['년도'].iloc[i]]  
    month = year[year['월']==check_zero['월'].iloc[i]]
    day = month[month['일']==check_zero['일'].iloc[i]]              
    dfr = pd.concat([dfr, day], axis=0)
    
dfr

,id,sub_id,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,년도,월,일,시간,발전량
3744,plant1,1,196.0,33.23831,126.34047,25.0,12.1,18.1,160.0,100.0,2018,8,23,4,0.0
3745,plant1,1,196.0,33.23831,126.34047,25.2,12.9,20.3,160.0,100.0,2018,8,23,5,0.0
3746,plant1,1,196.0,33.23831,126.34047,24.7,4.6,20.5,160.0,100.0,2018,8,23,6,0.0
3747,plant1,1,196.0,33.23831,126.34047,24.5,10.5,18.9,160.0,100.0,2018,8,23,7,0.0
3748,plant1,1,196.0,33.23831,126.34047,24.4,6.3,20.0,160.0,100.0,2018,8,23,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134939,plant9,1,40.0,35.06676,127.75295,11.6,0.0,3.3,270.0,29.0,2019,1,23,15,0.0
134940,plant9,1,40.0,35.06676,127.75295,10.8,0.0,2.5,290.0,30.0,2019,1,23,16,0.0
134941,plant9,1,40.0,35.06676,127.75295,9.6,0.0,5.3,290.0,28.0,2019,1,23,17,0.0
134942,plant9,1,40.0,35.06676,127.75295,7.9,0.0,4.1,290.0,27.0,2019,1,23,18,0.0


In [28]:
# 중복인 행 제거
dfr_index = dfr.index
sample = sample.drop(dfr_index)
    
sample = sample.reset_index(drop=True)
sample

,id,sub_id,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,년도,월,일,시간,발전량
0,plant1,1,196.0,33.23831,126.34047,6.2,0.0,9.5,340.0,56.0,2018,1,1,4,0.000
1,plant1,1,196.0,33.23831,126.34047,6.4,0.0,8.9,360.0,64.0,2018,1,1,5,0.000
2,plant1,1,196.0,33.23831,126.34047,5.9,0.0,8.7,360.0,58.0,2018,1,1,6,0.000
3,plant1,1,196.0,33.23831,126.34047,5.9,0.0,7.0,360.0,61.0,2018,1,1,7,0.000
4,plant1,1,196.0,33.23831,126.34047,5.8,0.0,8.1,360.0,59.0,2018,1,1,8,15.476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399211,plant21,1,13295.0,34.73819,126.41546,21.9,0.0,6.6,320.0,78.0,2020,6,30,15,3043.200
399212,plant21,1,13295.0,34.73819,126.41546,21.5,0.0,6.9,290.0,80.0,2020,6,30,16,3734.400
399213,plant21,1,13295.0,34.73819,126.41546,21.0,0.0,6.4,320.0,81.0,2020,6,30,17,1958.400
399214,plant21,1,13295.0,34.73819,126.41546,20.4,0.0,5.7,320.0,86.0,2020,6,30,18,662.400


In [30]:
# train, test 분할 각 plant별 상위 80 train
train = pd.DataFrame() 
test = pd.DataFrame()

for i in sample['id'].unique(): 
    for j in sample[sample['id'] == i]['sub_id'].unique():
        data = sample[(sample['id'] == i) &(sample['sub_id'] == j)]
        data['일수'] = 0 
        count = 1
        first = 0 
        for k in range(16,len(data)+16,16):
            data['일수'].iloc[first:k] = count
            count += 1 
            first += 16 
            
        n = data['일수'].max()
        
        train = train.append(data[0:int(n*0.8)*16])
        test = test.append(data[int(n*0.8)*16:])

<ipython-input-30-f778714b2fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'] = 0
/Users/chaeseungwan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-f778714b2fde>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'].iloc[first:k]

<ipython-input-30-f778714b2fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'] = 0
/Users/chaeseungwan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-f778714b2fde>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'].iloc[first:k]

<ipython-input-30-f778714b2fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'] = 0
/Users/chaeseungwan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-f778714b2fde>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'].iloc[first:k]

<ipython-input-30-f778714b2fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'] = 0
/Users/chaeseungwan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-f778714b2fde>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'].iloc[first:k]

<ipython-input-30-f778714b2fde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'] = 0
/Users/chaeseungwan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-30-f778714b2fde>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['일수'].iloc[first:k]

In [5]:
# train, test data load
import pandas as pd
train = pd.read_csv('./flask/app/data/train_0617.csv')
test = pd.read_csv('./flask/app/data/test_0617.csv')

In [6]:
train.head()

,id,sub_id,발전용량,위도,경도,기온,강수량,풍속,풍향,습도,년도,월,일,시,발전량,일수
0,plant1,1,196.0,33.23831,126.34047,6.2,0.0,9.5,340,56.0,2018,1,1,4,0.000,1
1,plant1,1,196.0,33.23831,126.34047,6.4,0.0,8.9,360,64.0,2018,1,1,5,0.000,1
2,plant1,1,196.0,33.23831,126.34047,5.9,0.0,8.7,360,58.0,2018,1,1,6,0.000,1
3,plant1,1,196.0,33.23831,126.34047,5.9,0.0,7.0,360,61.0,2018,1,1,7,0.000,1
4,plant1,1,196.0,33.23831,126.34047,5.8,0.0,8.1,360,59.0,2018,1,1,8,15.476,1


In [44]:
# LSTM 데이터 형태 맞춤
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index, step):
        indices = range(i-history_size, i)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In [124]:
# 단기모델
def solar_model(train_data,test_data,id,sub_id):
    tr_data = train_data[(train_data['id'] == id) & (train_data['sub_id'] == sub_id)].reset_index(drop=True)
    te_data = test_data[(test_data['id'] == id) & (test_data['sub_id'] == sub_id)].reset_index(drop=True)

    # 발전량 정규화
    capacity = tr_data['발전용량'][0]
    tr_data['value_nom'] = tr_data['발전량'] / capacity
    te_data['value_nom'] = te_data['발전량'] / capacity
    
    # 독립, 종속변수 설정 
    x_tr_data = tr_data[['기온','강수량','습도','풍속', '풍향', '시']]
    y_tr_data = tr_data[['value_nom']]
    x_te_data = te_data[['기온','강수량','습도','풍속', '풍향', '시']]
    y_te_data = te_data[['value_nom']]

    # 독립변수 정규화 
    min_max_scaler = MinMaxScaler()
    x_tr_data = min_max_scaler.fit_transform(x_tr_data)
    x_te_data = min_max_scaler.fit_transform(x_te_data)

    # y_data array형식으로 바꾸기
    y_tr_data = pd.DataFrame(y_tr_data)
    y_tr_data = y_tr_data.values
    y_te_data = pd.DataFrame(y_te_data)
    y_te_data = y_te_data.values

    # 데이터 형식 만들기 
    x_tr, y_tr = multivariate_data(x_tr_data, y_tr_data, 0,None,16,16,16)
    x_te, y_te = multivariate_data(x_te_data, y_te_data, 0,None,16,16,16)

    # 데이터 batch_size
    def gen():
        for i in zip(x_tr,y_tr):
            yield i
    m1 = tf.data.Dataset.from_generator(gen, output_shapes=(tf.TensorShape([None,None]),tf.TensorShape([None,None])), 
                                    output_types=(tf.float64, tf.float64))
    pad_batch = m1.batch(20)

    def gen2():
        for i in zip(x_te,y_te):
            yield i
 
    m2 = tf.data.Dataset.from_generator(gen2, output_shapes=(tf.TensorShape([None,None]),tf.TensorShape([None,None])), 
                                output_types=(tf.float64, tf.float64))
 
    test_pad_batch = m2.batch(20)

    # 모델
    inputs = tf.keras.layers.Input(shape=(16, 6))
    out1 = tf.keras.layers.LSTM(128, activation = 'relu', return_sequences=False)(inputs)
    #out2 = tf.keras.layers.LSTM(64, activation = 'relu', return_sequences=False)(out1)
    drop = tf.keras.layers.Dropout(0.2)(out1)
    out2 = tf.keras.layers.Dense(64)(drop)
    out3 = tf.keras.layers.Dense(16)(out2)
    model = tf.keras.Model(inputs = inputs, outputs = out3)
 
    model.compile(loss = 'mae',
                   optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
                                  #tf.keras.optimizers.RMSprop(clipvalue=1.0)
                ) 
                
 
 
    history = model.fit(pad_batch, epochs=30, verbose = 0)
    model.save('./flask/app/Short_model/'+str(id)+'_'+str(sub_id)+'.h5')

    # 예측값 
    y_pred = model.predict(x_te)
    
    # 오차율 구하기 
    yy_te = y_te.reshape(y_te.shape[0], 16)

    yy_te = yy_te * capacity
    y_pred = y_pred * capacity

    for i in range(len(yy_te)):
        for j in range(len(yy_te[i])):
            if yy_te[i][j] < capacity*0.1:
                yy_te[i][j] = 0
                y_pred[i][j] = 0 
                
    # 18, 19시 예측 값 0 으로 만들기
    for number in range(len(y_pred)):
        y_pred[number][14] = 0
        y_pred[number][15] = 0
                
    
    error = np.mean(abs(y_pred - yy_te) / capacity * 100)
    print(error)
    return error 

In [69]:
#장기 모델
def long_model(train_data,test_data,id,sub_id):
    tr_data = train_data[(train_data['id'] == id) & (train_data['sub_id'] == sub_id)].reset_index(drop=True)
    te_data = test_data[(test_data['id'] == id) & (test_data['sub_id'] == sub_id)].reset_index(drop=True)

    # 발전량 정규화
    capacity = tr_data['발전용량'][0]
    tr_data['value_nom'] = tr_data['발전량'] / capacity
    te_data['value_nom'] = te_data['발전량'] / capacity
    
    # 독립, 종속변수 설정 
    x_tr_data = tr_data[['기온','강수량','습도','풍속', '풍향', '시']]
    y_tr_data = tr_data[['value_nom']]
    x_te_data = te_data[['기온','강수량','습도','풍속', '풍향', '시']]
    y_te_data = te_data[['value_nom']]

    # 독립변수 정규화 
    min_max_scaler = MinMaxScaler()
    x_tr_data = min_max_scaler.fit_transform(x_tr_data)
    x_te_data = min_max_scaler.fit_transform(x_te_data)

    # y_data array형식으로 바꾸기
    y_tr_data = pd.DataFrame(y_tr_data)
    y_tr_data = y_tr_data.values
    y_te_data = pd.DataFrame(y_te_data)
    y_te_data = y_te_data.values

    # 데이터 형식 만들기 
    x_tr, y_tr = multivariate_data(x_tr_data, y_tr_data, 0,None,16,160,16)
    x_te, y_te = multivariate_data(x_te_data, y_te_data, 0,None,16,160,16)

    # 데이터 batch_size
    def gen():
        for i in zip(x_tr,y_tr):
            yield i
    m1 = tf.data.Dataset.from_generator(gen, output_shapes=(tf.TensorShape([None,None]),tf.TensorShape([None,None])), 
                                    output_types=(tf.float64, tf.float64))
    pad_batch = m1.batch(20)

    def gen2():
        for i in zip(x_te,y_te):
            yield i
 
    m2 = tf.data.Dataset.from_generator(gen2, output_shapes=(tf.TensorShape([None,None]),tf.TensorShape([None,None])), 
                                output_types=(tf.float64, tf.float64))
 
    test_pad_batch = m2.batch(20)

    # 모델
    inputs = tf.keras.layers.Input(shape=(16, 6))
    out1 = tf.keras.layers.LSTM(128, activation = 'relu', return_sequences=False)(inputs)
    #out2 = tf.keras.layers.LSTM(64, activation = 'relu', return_sequences=False)(out1)
    drop = tf.keras.layers.Dropout(0.2)(out1)
    out2 = tf.keras.layers.Dense(64)(drop)
    out3 = tf.keras.layers.Dense(160)(out2)
    model = tf.keras.Model(inputs = inputs, outputs = out3)
 
    model.compile(loss = 'mae',
                   optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
                                  #tf.keras.optimizers.RMSprop(clipvalue=1.0)
                ) 
                
 
 
    history = model.fit(pad_batch, epochs=30, verbose = 0)
    model.save('./flask/app/long_model/'+str(id)+'_'+str(sub_id)+'.h5')

    # 예측값 
    y_pred = model.predict(x_te)
    
    # 오차율 구하기 
    yy_te = y_te.reshape(y_te.shape[0], 160)

    yy_te = yy_te * capacity
    y_pred = y_pred * capacity

    for i in range(len(yy_te)):
        for j in range(len(yy_te[i])):
            if yy_te[i][j] < capacity*0.1:
                yy_te[i][j] = 0
                y_pred[i][j] = 0 
                
                
    error = np.mean(abs(y_pred - yy_te) / capacity * 100)
    print(error)
    return error 

In [70]:
for i in train['id'].unique():
    for j in train.loc[train['id']==i]['sub_id'].unique():
        #print('id :', i, 'sub_id :', j)
        solar_model(train, test, i, j)

5.932878195683118
8.316146120428373
1.556114966153048
6.707009707917053
5.955310988832471
8.848370917466154
8.04635593125317
6.490100288323655
7.562245905569942
7.784767431361977
6.415597964460242
6.55223138355724
7.222629308938559
7.905583214906848
8.889493226685655
7.639283422553094
6.510618240836268
7.851253239931456
8.798902629660502
9.576394584139075
10.192951833090204
10.26278496211182
7.225787726153594
6.7296730339524755
10.928114058708495
10.07440720887639
6.9741979818378566
8.468889961226376
8.394735555844365
8.123081679896012
6.7287890063575695
5.42681132349956
7.340798114665321
7.05708996187222
6.5504149376340735
8.043145498185492


7.673463063558478
6.655044319345552
8.601986815433483


In [ ]:
for i in train['id'].unique():
    for j in train.loc[train['id']==i]['sub_id'].unique():
        long_model(train, test, i, j)